In [1]:
import osmosys.osmosys
import activityinfo
from activityinfo import Client
from activityinfo import FormProcessing
import importlib
import model.modelAI
from activityinfo.id import generate_id
import json
from requests.exceptions import HTTPError
import os.path

importlib.reload(osmosys.osmosys)
importlib.reload(activityinfo.FormProcessing)
importlib.reload(model.modelAI)

<module 'model.modelAI' from 'D:\\Proyectos\\OSMOSYS_2022\\IntegraciónAI\\AI_OsmosysIntegrationScripts\\model\\modelAI.py'>

In [3]:
## parameters
indicatorCodeAI = 'PR2_02'
indicatorCodeOsmosys = 'AE0I1'
month = 'JUNIO'
month_number = '2022-06'
year = 2022
print('--------------------------------------PR2_02------------------------------------------------------')

--------------------------------------PR2_02------------------------------------------------------


In [4]:
# busco matchs
subformDf = osmosys.osmosys.getMatchSubforms(indicatorCodeAI)
## print(subformDf)
indicatorIdsOsmosys = subformDf.osmosys_indicador_id.unique()
subformSeries = subformDf.iloc[0]
formIdAI = subformSeries.form_id
parentFormIdAI = subformSeries.parent_form_id
indicatorIdAI = subformSeries.indicator_id
print("formIdAI: " + formIdAI)
print("parentFormIdAI: " + parentFormIdAI)
print("indicatorIdAI: " + indicatorIdAI)
print("indicatorIdsOsmosys: " + str(indicatorIdsOsmosys))

formIdAI: cxsom3tl4q3ommxdi
parentFormIdAI: cq0tf6fl4q3ommxdu
indicatorIdAI: cw13jl0kylzqu4i43
indicatorIdsOsmosys: [93]


In [5]:
## consulto los datos reportados
reportedDf = osmosys.osmosys.getIesPartnerCantonsByIndicatorsIdsOsmosysAndMonth(indicatorsIdsOmosys=indicatorIdsOsmosys,
                                                                                month=month, year=year)
print('reported data: '+str(reportedDf.shape[0]))


reported data: 26


In [6]:
## obtengo los parents de AI
parentFormsIds = FormProcessing.getParentsIds(parentFormIdAI=parentFormIdAI, reportedDf=reportedDf)


In [8]:
## contruyo las estructuras de datos
changesList = []
for index, row in parentFormsIds.iterrows():
    # print(row)
    orgAcron = row.acronym
    cantonCode = row.canton_code
    df = osmosys.osmosys.getRefValues(year=year, month=month, orgOsmosys=orgAcron,
                                      indicatorsIdsOmosys=indicatorIdsOsmosys, cantonCode=cantonCode)
    r = df.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a

    PR2_02_RM_NA = int(df.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    PR2_02_RM_NN = int(df.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    PR2_02_RM_MM = int(df.loc[df['age_gender'] == 'ADULTOS'].iloc[0].value_a)
    PR2_02_RM_HH = int(df.loc[df['age_gender'] == 'ADULTAS'].iloc[0].value_a)
    PR2_02_RM_OTR = int(df.loc[df['age_gender'] == 'OTRO'].iloc[0].value_a)
    dfDiversidad = osmosys.osmosys.getRefLgbtiDiscapacitadosValues(year=year, month=month, orgOsmosys=orgAcron,
                                                                   indicatorsIdsOmosys=indicatorIdsOsmosys,
                                                                   cantonCode=cantonCode)
    #print(dfDiversidad)
    PR2_02_RM_LGBT = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'LGBTI') & (
            dfDiversidad['country_of_origin'] == 'VENEZUELA')].iloc[0].value_a)
    PR2_02_RM_DS = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'DISCAPACITADOS') & (
            dfDiversidad['country_of_origin'] == 'VENEZUELA')].iloc[0].value_a)
    commentary = osmosys.osmosys.getCommentary(year=year, month=month, orgOsmosys=orgAcron,
                                               indicatorsIdsOmosys=indicatorIdsOsmosys).iloc[0].value_a
    PR2_02_CUAL = commentary
    modalidad_impl = "cmo12q1kykwwosjka"

    subform = model.modelAI.SubFormPR2_02(mes=month_number, colltmgkykvhxgij6=indicatorIdAI, rmrp='Si', covid='No',
                                          espacion_apoyo='No', poblacion_meta='Refugiados/as y migrantes',
                                          modalidad_impl=modalidad_impl, PR2_02_RM_NA=PR2_02_RM_NA,
                                          PR2_02_RM_NN=PR2_02_RM_NN,
                                          PR2_02_RM_MM=PR2_02_RM_MM, PR2_02_RM_HH=PR2_02_RM_HH,
                                          PR2_02_RM_OTR=PR2_02_RM_OTR, PR2_02_RM_LGBT=PR2_02_RM_LGBT,
                                          PR2_02_RM_DS=PR2_02_RM_DS, PR2_02_CUAL=PR2_02_CUAL)

    record = model.modelAI.Record(formId=formIdAI, recordId=generate_id(), parentRecordId=row['@id'], fields=subform)
    changesList.append(record)
print('changes to charge: '+str(len(changesList)))

changes to charge: 26


In [11]:
## creo respaldo
changes = model.modelAI.Changes(changesList)
finalJson = json.dumps(changes, default=model.modelAI.default)
#open text file
try:
    os.mkdir(month)
except Exception:
    pass
text_file = open(os.path.join(month,"data_" + indicatorCodeAI + ".json"), "w")
#write string to file
n = text_file.write(finalJson)

#close file
text_file.close()
print(" creado respaldo: "+text_file.name)

 creado respaldo: JUNIO\data_PR2_02.json


In [ ]:
## envio a AI
try:
    if len(changesList)>0:
        print('se envia a AI')
        client = Client(token=osmosys.osmosys.getToken(), base_url='https://www.activityinfo.org/resources')
        client.post_resource(path='update', body=finalJson)
    else:
        print('nada que enviar a AI')
except HTTPError as e:
    code = e.response.status_code
    print('error:')
    print(code)

    print(e)